In [1]:
import os
from typing import List
import json
import argparse
import logging
import pandas as pd
from tqdm import tqdm

In [2]:
import sys
sys.path.append("../")

In [3]:
from src.hipporag import HippoRAG
from dotenv import load_dotenv
load_dotenv()

/Users/taindp/miniconda3/envs/hipporag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-20 14:16:50,289	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


True

In [4]:
## clean cache
import shutil
shutil.rmtree("outputs", ignore_errors=True)

In [5]:
corpus_path = "../data/courses_DS.json"
with open(corpus_path, "r") as f:
    corpus = json.load(f)

docs = [f"{doc['title']}\n{doc['text']}" for doc in corpus]

save_dir = 'outputs/openai_test'  # Define save directory for HippoRAG objects (each LLM/Embedding model combination will create a new subdirectory)
llm_model_name = 'gpt-4o-mini'  # Any OpenAI model name
embedding_model_name = 'text-embedding-3-small'  # Embedding model name (NV-Embed, GritLM or Contriever for now)

# Startup a HippoRAG instance
hipporag = HippoRAG(save_dir=save_dir,
                    llm_model_name=llm_model_name,
                    embedding_model_name=embedding_model_name,
                    dataset="hcmus" ## HippoRAG base
                    )

# Run indexing
hipporag.index(docs=docs)

Batch Encoding: 80it [00:06, 11.90it/s]                        
Extracting triples: 100%|██████████| 75/75 [01:02<00:00,  1.20it/s, total_prompt_tokens=66041, total_completion_tokens=28941, num_cache_hit=0]
Batch Encoding: 1344it [01:17, 17.35it/s]                          
Batch Encoding: 1664it [01:26, 19.22it/s]                          
75it [00:00, 18587.38it/s]
75it [00:00, 52446.28it/s]
100%|██████████| 1339/1339 [00:00<00:00, 72697.51it/s]

{'num_phrase_nodes': 1339, 'num_passage_nodes': 75, 'num_total_nodes': 1414, 'num_extracted_triples': 1654, 'num_triples_with_passage_node': 1797, 'num_synonymy_triples': 3201, 'num_total_triples': 6652}


In [6]:
# docs

In [7]:
open_end_qa_ds = pd.DataFrame(json.load(open("../data/open_end_qa_ds.json", "r")))
queries = open_end_qa_ds["question"].tolist()
references = open_end_qa_ds["answer"].tolist()
gold_docs = [[f"{item[0]['title']}\n{item[0]['text']}"] for item in open_end_qa_ds["paragraphs"].tolist()]
# gold_docs

In [8]:
# gold_docs

In [9]:
queries_solutions, all_response_message, all_metadata, overall_retrieval_result, overall_qa_results = hipporag.rag_qa(
    queries=queries,
    gold_docs=gold_docs,
    gold_answers=references
)
# predictions = [item.split("Answer: ")[1] for item in all_response_message]

Batch Encoding: 32it [00:01, 25.56it/s]                        
Batch Encoding: 32it [00:01, 22.38it/s]                        
Retrieving: 100%|██████████| 25/25 [00:51<00:00,  2.06s/it]
Length of retrieved docs (75) is smaller than largest topk for recall score (200)
Length of retrieved docs (75) is smaller than largest topk for recall score (200)
Length of retrieved docs (75) is smaller than largest topk for recall score (200)
Length of retrieved docs (75) is smaller than largest topk for recall score (200)
Length of retrieved docs (75) is smaller than largest topk for recall score (200)
Length of retrieved docs (75) is smaller than largest topk for recall score (200)
Length of retrieved docs (75) is smaller than largest topk for recall score (200)
Length of retrieved docs (75) is smaller than largest topk for recall score (200)
Length of retrieved docs (75) is smaller than largest topk for recall score (200)
Length of retrieved docs (75) is smaller than largest topk for recall scor

In [10]:
overall_retrieval_result

{'Recall@1': 0.36,
 'Recall@2': 0.52,
 'Recall@5': 0.68,
 'Recall@10': 0.72,
 'Recall@20': 0.72,
 'Recall@30': 0.72,
 'Recall@50': 0.72,
 'Recall@100': 0.72,
 'Recall@150': 0.72,
 'Recall@200': 0.72}